In [42]:
import pandas as pd
import numpy as np
import math
import urllib
import requests
from bs4 import BeautifulSoup as BS

In [2]:
tickers = ['MNTA', 'AON', 'OPHT', 'IIVI', 'MMM', 'GE']
input_type = 'DEF+14'


In [3]:
url_list = []

for i in range(0, len(tickers)):
    url_list.append('https://www.sec.gov/cgi-bin/browse-edgar?CIK=' + tickers[i] + '&owner=include&action=getcompany&type=' + input_type)    
url_list

['https://www.sec.gov/cgi-bin/browse-edgar?CIK=MNTA&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=AON&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=OPHT&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=IIVI&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=MMM&owner=include&action=getcompany&type=DEF+14',
 'https://www.sec.gov/cgi-bin/browse-edgar?CIK=GE&owner=include&action=getcompany&type=DEF+14']

In [16]:
#dont need
SEC_List = pd.DataFrame()
SEC_List = pd.read_html(url_list[0])
filing_list = SEC_List[2]
filing_list.rename(columns = filing_list.iloc[0], inplace = True)
filing_list = filing_list.iloc[1:]
filing_list.reset_index(inplace = True)
filing_list

,index,Filings,Format,Description,Filing Date,File/Film Number
0,1,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2018-04-27,000-5079718783728
1,2,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00012...,2017-04-27,000-5079717786330
2,3,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2016-04-27,000-50797161595941
3,4,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2015-04-24,000-5079715791019
4,5,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2014-04-22,000-5079714775778
5,6,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2013-04-22,000-5079713773429
6,7,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2012-04-23,000-5079712772759
7,8,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2011-04-26,000-5079711780124
8,9,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2010-04-27,000-5079710773037
9,10,DEF 14A,Documents,Other definitive proxy statementsAcc-no: 00010...,2009-04-30,000-5079709780825


In [24]:
#don't need
ACC_Num = []

for i in range(0,filing_list.shape[0]):
    start_num = filing_list['Description'][i].find(':') + 2
    end_num = filing_list['Description'][i].find("\xa0")
    ACC_Num.append(filing_list['Description'][i][start_num:end_num])

ACC_Num

['0001047469-18-003290',
 '0001235010-17-000006',
 '0001047469-16-012622',
 '0001047469-15-003889',
 '0001047469-14-004032',
 '0001047469-13-004657',
 '0001047469-12-004605',
 '0001047469-11-004077',
 '0001047469-10-004249',
 '0001047469-09-004743',
 '0001047469-08-005475',
 '0001104659-07-032586',
 '0001104659-06-028782',
 '0001047469-05-010762']

In [91]:
#BS grabs links here
tag_list = []
page = requests.get(url_list[0])

soup = BS(page.content, 'lxml')

doc_link = soup.find_all('a')

for i in range(0,len(doc_link)):
    x = doc_link[i].get('href')
    y = x.split('/')
    if len(y) > 1:
        if y[1] == 'Archives':
            tag_list.append(x)

document_links = []            
            
for i in range(0,len(tag_list)):
    index_page = requests.get('https://www.sec.gov'+tag_list[i])
    soup2= BS(index_page.content, 'lxml')
    doc_link2 = soup2.find_all('a')
    document_links.append(doc_link2[8].get('href'))
    
document_links

['/Archives/edgar/data/1235010/000104746918003290/a2235381zdef14a.htm',
 '/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm',
 '/Archives/edgar/data/1235010/000104746916012622/a2228452zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746915003889/a2224436zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746914004032/a2219693zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746913004657/a2214528zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746912004605/a2208989zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746911004077/a2203592zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746910004249/a2198205zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746909004743/a2192562zdef14a.htm',
 '/Archives/edgar/data/1235010/000104746908005475/a2185071zdef14a.htm',
 '/Archives/edgar/data/1235010/000110465907032586/a07-6436_1def14a.htm',
 '/Archives/edgar/data/1235010/000110465906028782/a06-9098_1def14a.htm',
 '/Archives/edgar/data/1235010/000104746905010762/a2156

In [92]:
filing_reader = []
for i in range(0,len(document_links)):
    url = 'https://www.sec.gov'+document_links[i]
    filing_reader.append(pd.read_html(url))
    


In [93]:
filing_reader[0]

[                                              0   1  \
 0                                           NaN NaN   
 1                     Filed by the Registrant ý NaN   
 2  Filed by a Party other than the Registrant o NaN   
 3                    Check the appropriate box: NaN   
 4                                             o NaN   
 5                                             o NaN   
 6                                             ý NaN   
 7                                             o NaN   
 8                                             o NaN   
 
                                                    2  
 0                                                NaN  
 1                                                NaN  
 2                                                NaN  
 3                                                NaN  
 4                        Preliminary Proxy Statement  
 5  Confidential, for Use of the Commission Only (...  
 6                         Definitive Proxy St

In [114]:
filing_reader[0][21]

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name and Principal Position,NaN,Year,NaN,Salary ($),NaN,Stock Awards(1) ($),NaN,Option Awards(1) ($),NaN,...,NaN,Total ($),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Craig A. Wheeler,NaN,NaN,2017.0,NaN,NaN,750000,NaN,NaN,1743625,...,NaN,NaN,480000,NaN,NaN,34551.0,NaN,NaN,4784639.0,NaN
3,"President, Chief Executive",NaN,NaN,2016.0,NaN,NaN,689179,NaN,NaN,2149290,...,NaN,NaN,554789,NaN,NaN,29948.0,NaN,NaN,4161024.0,NaN
4,Officer and Director,NaN,NaN,2015.0,NaN,NaN,691484,NaN,NaN,3064900,...,NaN,NaN,512723,NaN,NaN,29503.0,NaN,NaN,5344662.0,NaN
5,Scott M. Storer,NaN,NaN,2017.0,NaN,NaN,420000,NaN,NaN,—,...,NaN,NaN,137760,NaN,NaN,10848.0,NaN,NaN,568608.0,NaN
6,Senior Vice President and Chief,NaN,NaN,2016.0,NaN,NaN,40385,NaN,NaN,684000,...,NaN,NaN,—,NaN,NaN,150.0,NaN,NaN,1664910.0,NaN
7,Financial Officer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Ganesh V. Kaundinya,NaN,NaN,2017.0,NaN,NaN,448517,NaN,NaN,425068,...,NaN,NaN,196800,NaN,NaN,13481.0,NaN,NaN,1516939.0,NaN
9,Chief Operating Officer and,NaN,NaN,2016.0,NaN,NaN,433350,NaN,NaN,791487,...,NaN,NaN,201833,NaN,NaN,13091.0,NaN,NaN,1661106.0,NaN
